In [1]:
pip install praw textblob yfinance scikit-learn pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import praw

In [3]:
import pandas as pd

In [5]:
from textblob import TextBlob

# Step 1: Reddit API Setup

In [7]:
reddit = praw.Reddit(
    client_id="40JwTJ-pMvk1w_Uv_3DYYQ",             # Replace with your Reddit client ID
    client_secret="wQzYby_ImWc3TqIiV-mYiZyIc-5JPg",     # Replace with your Reddit client secret
    user_agent="Arnav:1.0 (by u/WoodpeckerRich2663)"            # Replace with your user agent
)

In [8]:
print(reddit.read_only)

True


# Step 2: Scrape Data from a Subreddit

In [11]:
def scrape_reddit(subreddit_name, keyword, limit=100):
    subreddit = reddit.subreddit(subreddit_name)
    data = []
    for submission in subreddit.search(keyword, limit=limit):
        data.append({
            "title": submission.title,
            "created_utc": pd.to_datetime(submission.created_utc, unit="s"),
            "score": submission.score,
            "comments": submission.num_comments,
            "upvote_ratio": submission.upvote_ratio
        })
    return pd.DataFrame(data)


In [12]:
# Scrape data from a stock-related subreddit (e.g., r/stocks)
reddit_data = scrape_reddit("stocks", "AAPL", limit=200)
print("Scraped Reddit data:")
print(reddit_data.head())

Scraped Reddit data:
                                               title         created_utc  \
0             Are you worried about AAPL long term?  2024-05-08 14:07:32   
1                                       MSFT or AAPL 2024-10-01 04:54:10   
2  I have $213,000 in Apple (AAPL) stock, should ... 2020-10-13 09:19:24   
3  China plans to expand its ban on iPhones, AAPL... 2023-09-07 13:15:44   
4  The U.S. Department of Justice (DOJ) is prepar... 2024-03-21 11:32:28   

   score  comments  upvote_ratio  
0    349       456          0.78  
1      0        83          0.40  
2   2454       727          0.93  
3    774       399          0.94  
4    612       212          0.95  


In [13]:
reddit_data.shape

(200, 5)

In [14]:
reddit_data.columns

Index(['title', 'created_utc', 'score', 'comments', 'upvote_ratio'], dtype='object')

In [16]:
reddit_data['title']

0                 Are you worried about AAPL long term? 
1                                           MSFT or AAPL
2      I have $213,000 in Apple (AAPL) stock, should ...
3      China plans to expand its ban on iPhones, AAPL...
4      The U.S. Department of Justice (DOJ) is prepar...
                             ...                        
195                               This is NOT the end...
196             What happened to AAPL after hours today?
197              Buy AAPL as a forever hold now or wait?
198    r/Stocks Daily Discussion & Fundamentals Frida...
199             AAPL/MSFT/GOOG/AMZN Growth Expectations?
Name: title, Length: 200, dtype: object

# Step 3: Perform Sentiment Analysis

In [17]:
def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # Returns a float between -1.0 (negative) and 1.0 (positive)

In [25]:
reddit_data["sentiment"] = reddit_data["title"].apply(analyze_sentiment)
print("\nSentiment analysis added:")
print(reddit_data[["title", "sentiment"]].head(10))


Sentiment analysis added:
                                               title  sentiment
0             Are you worried about AAPL long term?      -0.050
1                                       MSFT or AAPL      0.000
2  I have $213,000 in Apple (AAPL) stock, should ...      0.000
3  China plans to expand its ban on iPhones, AAPL...      0.000
4  The U.S. Department of Justice (DOJ) is prepar...     -0.100
5                            AAPL is at 52-week low!      0.000
6   AAPL reaches all time high in pre-market trading      0.160
7  Whats your "this could be the next tsla or aap...      0.000
8                                     I SOLD AAPL :(     -0.750
9  Buy AAPL in front of 4:1 stock split. Stock sp...      0.125


# Step 4: Download Historical Stock Data

In [33]:
import yfinance as yf
stock_data = yf.download("AAPL", start="2018-01-01", end="2024-11-26")
print("\nDownloaded stock data:")
print(stock_data.head())

[*********************100%***********************]  1 of 1 completed


Downloaded stock data:
Price       Adj Close      Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                                        
2018-01-02  40.524349  43.064999  43.075001  42.314999  42.540001  102223600
2018-01-03  40.517284  43.057499  43.637501  42.990002  43.132500  118071600
2018-01-04  40.705494  43.257500  43.367500  43.020000  43.134998   89738400
2018-01-05  41.168930  43.750000  43.842499  43.262501  43.360001   94640000
2018-01-08  41.016026  43.587502  43.902500  43.482498  43.587502   82271200


In [34]:
# Ensure 'Date' column is present in the Reddit data and aligns with stock data
reddit_data["date"] = reddit_data["created_utc"].dt.date
stock_data.reset_index(inplace=True)
stock_data["date"] = stock_data["Date"].dt.date

In [41]:
reddit_data.sort_values('date')

,title,created_utc,score,comments,upvote_ratio,sentiment,date
178,"$AAPL up almost 20% since February, thoughts o...",2018-06-05 15:31:34,103,78,0.86,0.000000,2018-06-05
117,Apple (#AAPL) now only -31% since Warren Buffe...,2018-12-21 02:38:11,124,108,0.89,0.000000,2018-12-21
49,QCOM Vs. AAPL settlement. How I lost the oppor...,2019-04-16 22:59:52,367,171,0.91,-1.000000,2019-04-16
144,More AAPL for me please,2019-05-24 19:20:08,98,85,0.90,0.500000,2019-05-24
18,Microsoft (MSFT) and Apple (AAPL) are Now Bigg...,2019-12-28 10:48:33,1202,264,0.97,0.000000,2019-12-28
...,...,...,...,...,...,...,...
35,Warren Buffett’s Berkshire Hathaway sold ~$14 ...,2024-11-02 12:19:31,1049,333,0.96,0.300000,2024-11-02
157,r/Stocks Daily Discussion & Fundamentals Frida...,2024-11-08 10:30:10,16,413,0.84,0.000000,2024-11-08
137,r/Stocks Daily Discussion & Fundamentals Frida...,2024-11-15 10:30:09,23,466,0.89,0.000000,2024-11-15
198,r/Stocks Daily Discussion & Fundamentals Frida...,2024-11-22 10:30:12,14,253,0.83,0.000000,2024-11-22


In [42]:
stock_data

Price,Date,Adj Close,Close,High,Low,Open,Volume,date
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,
0,2018-01-02,40.524349,43.064999,43.075001,42.314999,42.540001,102223600,2018-01-02
1,2018-01-03,40.517284,43.057499,43.637501,42.990002,43.132500,118071600,2018-01-03
2,2018-01-04,40.705494,43.257500,43.367500,43.020000,43.134998,89738400,2018-01-04
3,2018-01-05,41.168930,43.750000,43.842499,43.262501,43.360001,94640000,2018-01-05
4,2018-01-08,41.016026,43.587502,43.902500,43.482498,43.587502,82271200,2018-01-08
...,...,...,...,...,...,...,...,...
1732,2024-11-19,228.279999,228.279999,230.160004,226.660004,226.979996,36211800,2024-11-19
1733,2024-11-20,229.000000,229.000000,229.929993,225.889999,228.059998,35169600,2024-11-20
1734,2024-11-21,228.520004,228.520004,230.160004,225.710007,228.880005,42108300,2024-11-21


In [49]:
reddit_data.columns

Index(['title', 'created_utc', 'score', 'comments', 'upvote_ratio',
       'sentiment', 'date'],
      dtype='object')

In [50]:
stock_data.columns

MultiIndex([(     'Date',     ''),
            ('Adj Close', 'AAPL'),
            (    'Close', 'AAPL'),
            (     'High', 'AAPL'),
            (      'Low', 'AAPL'),
            (     'Open', 'AAPL'),
            (   'Volume', 'AAPL'),
            (     'date',     '')],
           names=['Price', 'Ticker'])

In [55]:
stock_data.columns=stock_data.columns.get_level_values(0)

In [56]:
stock_data.columns

Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'date'], dtype='object', name='Price')

# Step 5: Merge Reddit Data with Stock Data

In [64]:
merged_data = pd.merge(reddit_data, stock_data[["date", "Close"]], on="date", how='inner')
print("\nMerged data:")
print(merged_data.head())


Merged data:
                                               title         created_utc  \
0             Are you worried about AAPL long term?  2024-05-08 14:07:32   
1                                       MSFT or AAPL 2024-10-01 04:54:10   
2  I have $213,000 in Apple (AAPL) stock, should ... 2020-10-13 09:19:24   
3  China plans to expand its ban on iPhones, AAPL... 2023-09-07 13:15:44   
4  The U.S. Department of Justice (DOJ) is prepar... 2024-03-21 11:32:28   

   score  comments  upvote_ratio  sentiment        date       Close  
0    349       456          0.78      -0.05  2024-05-08  182.740005  
1      0        83          0.40       0.00  2024-10-01  226.210007  
2   2454       727          0.93       0.00  2020-10-13  121.099998  
3    774       399          0.94       0.00  2023-09-07  177.559998  
4    612       212          0.95      -0.10  2024-03-21  171.369995  


In [65]:
merged_data.columns

Index(['title', 'created_utc', 'score', 'comments', 'upvote_ratio',
       'sentiment', 'date', 'Close'],
      dtype='object')

In [66]:
merged_data.shape

(141, 8)

# Step 6: Create a Target Variable


In [68]:
merged_data1=merged_data.sort_values('date')

In [69]:
merged_data1

,title,created_utc,score,comments,upvote_ratio,sentiment,date,Close
126,"$AAPL up almost 20% since February, thoughts o...",2018-06-05 15:31:34,103,78,0.86,0.0,2018-06-05,48.327499
80,Apple (#AAPL) now only -31% since Warren Buffe...,2018-12-21 02:38:11,124,108,0.89,0.0,2018-12-21,37.682499
37,QCOM Vs. AAPL settlement. How I lost the oppor...,2019-04-16 22:59:52,367,171,0.91,-1.0,2019-04-16,49.812500
100,More AAPL for me please,2019-05-24 19:20:08,98,85,0.90,0.5,2019-05-24,44.742500
91,AAPL suspends 2020 iPhone model production,2020-04-27 11:50:28,95,79,0.92,0.0,2020-04-27,70.792503
...,...,...,...,...,...,...,...,...
133,r/Stocks Daily Discussion & Fundamentals Frida...,2024-09-06 09:30:09,16,457,0.78,0.0,2024-09-06,220.820007
1,MSFT or AAPL,2024-10-01 04:54:10,0,83,0.40,0.0,2024-10-01,226.210007
111,r/Stocks Daily Discussion & Fundamentals Frida...,2024-11-08 10:30:10,16,413,0.84,0.0,2024-11-08,226.960007
94,r/Stocks Daily Discussion & Fundamentals Frida...,2024-11-15 10:30:09,23,466,0.89,0.0,2024-11-15,225.000000


In [111]:
# Predict whether the stock price will increase (1) or decrease (0) the next day
merged_data1["target"] = (merged_data1["Close"]>merged_data1["Close"].shift(1)).astype(int)
merged_data1.dropna(inplace=True)  # Drop rows with NaN values caused by shifting
print("\nMerged data with target variable:")
print(merged_data1.head(20))


Merged data with target variable:
                                                 title         created_utc  \
126  $AAPL up almost 20% since February, thoughts o... 2018-06-05 15:31:34   
80   Apple (#AAPL) now only -31% since Warren Buffe... 2018-12-21 02:38:11   
37   QCOM Vs. AAPL settlement. How I lost the oppor... 2019-04-16 22:59:52   
100                            More AAPL for me please 2019-05-24 19:20:08   
91          AAPL suspends 2020 iPhone model production 2020-04-27 11:50:28   
89                                      I did it today 2020-06-09 20:42:32   
19   The moment AAPL announced ending partnership w... 2020-06-22 19:06:10   
119             Why is it that some people avoid AAPL? 2020-07-01 08:24:21   
138            Buy AAPL as a forever hold now or wait? 2020-08-07 21:12:06   
9    Buy AAPL in front of 4:1 stock split. Stock sp... 2020-08-07 15:58:10   
62   How did AAPL gain close to 1 trillion in marke... 2020-08-11 21:43:23   
28   Epic games sues Apple. W

# Step 7: Prepare Data for Machine Learning

In [127]:
from sklearn.model_selection import train_test_split
features = ["sentiment", "score", "comments", "upvote_ratio"]
X = merged_data1[features]
y = merged_data1["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 8: Train a Machine Learning Model

In [128]:
from sklearn.ensemble import RandomForestClassifier

In [129]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# Step 9: Evaluate the Model

In [130]:
from sklearn.metrics import classification_report

In [131]:
y_pred = model.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.26      0.37        19
           1       0.33      0.70      0.45        10

    accuracy                           0.41        29
   macro avg       0.48      0.48      0.41        29
weighted avg       0.52      0.41      0.40        29



# Step 10: Save the Results

In [132]:
merged_data1.to_csv("merged_stock_sentiment_data.csv", index=False)
print("\nResults saved to 'merged_stock_sentiment_data.csv'")


Results saved to 'merged_stock_sentiment_data.csv'
